# SVM 알고리즘
## 서포트 벡터 머신

In [ ]:
'''
SVM (Support Vector Machine) : 
 - 학습방법 : 벡터 공간에 위치한 훈련 데이터의 좌표와 각 데이터가 어떤 분류값을 가져야하는지 
             레이블을 입력받아 처리하는 형태
 
 - 설명변수의 열 벡터는 각각의 고유의 축을 갖는 벡터 공간을 만들 수 있고,
    분석 대상이 되는 개별 관측값은 모든 속성(열 벡터)에 관한 값을 
    해당 축의 좌표로 표시하여 벡터 공간에서의 위치로 나타내는 방법
 
 - 만약 열 벡터가 2개 존재하는 데이터 세트라면 2차원 평면 공간, 
   3개 이상이라면 3차원, 4개 이상이면 고차원 벡터 공간 좌표를 사용
'''

In [1]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')
df.head()



,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
rdf = df.drop(['deck', 'embark_town'], axis=1)

rdf = rdf.dropna(subset=['age'], how='any', axis=0)

freq = rdf['embarked'].value_counts(dropna=True).idxmax()

rdf['embarked'].fillna(freq, inplace=True)
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='embarked')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'], axis=1, inplace=True)

X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'embarked_C', 'embarked_Q', 'embarked_S']]
y = ndf[['survived']]

from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit(X).transform(X)
X.shape

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print(X_train.shape, ' : ', X_test.shape)

(571, 9)  :  (143, 9)


### SVM 분류 모형 학습/예측/평가

In [4]:
from sklearn.svm import SVC

# kernel : 데이터를 벡터 공간으로 매핑해주는 함수
# rbf(radal basic function - default) | linear | Polynamal | Sigmoid

svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC()

In [6]:
y_pred = svm_model.predict(X_test)
print(y_pred[:10])

print('-'*50)
print(y_test.values[:10])

[0 0 1 0 0 1 1 0 0 0]
--------------------------------------------------
[[0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]]


In [7]:
# 성능평가

from sklearn.metrics import classification_report
svm_report = classification_report(y_test, y_pred)
print(svm_report)


# 같은 데이터셋으로 알고리즘만 바꾸었는데 평가 지표들이 더 좋게 나옴.
# ==> 더 좋게 나오는 알고리즘을 찾아 쓰면 된다

              precision    recall  f1-score   support

           0       0.78      0.94      0.86        85
           1       0.88      0.62      0.73        58

    accuracy                           0.81       143
   macro avg       0.83      0.78      0.79       143
weighted avg       0.82      0.81      0.80       143



# 결정 트리(Decision Tree)

In [8]:
import pandas as pd
import numpy as np

In [20]:
# 데이터 로딩
uci_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'

df = pd.read_csv(uci_data, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [12]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,10
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4
698,897471,4,8,8,5,4,5,10,4,1,4


In [ ]:
'''
dataset 설명 - 유방암 세포 조직의 크기와 모양 등 종양 특성을 타나내는 열 9개, 마지막 열 1개는 악성 종양 여부(2: 양성, 4: 악성) 


'''

In [13]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       699 non-null    int64 
 1   1       699 non-null    int64 
 2   2       699 non-null    int64 
 3   3       699 non-null    int64 
 4   4       699 non-null    int64 
 5   5       699 non-null    int64 
 6   6       699 non-null    object
 7   7       699 non-null    int64 
 8   8       699 non-null    int64 
 9   9       699 non-null    int64 
 10  10      699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [14]:
df.columns = ['id', 'clump', 'cell_size', 'cell_shape', 'adhesion', 'epithlial', 
              'bare_nucleoli', 'chromatin', 'normal_nucleoli', 'mitoses', 'class']

df.head()

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nucleoli,chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [15]:
# info()로 확인해보니 오브젝트 자료형이 들어간 열이 있으므로
# 그 열에 대해서 특이한 데이터를 확인

df['bare_nucleoli'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [17]:
# unique() 확인 결과 '?' 가 들어가있으므로 전처리 수행

df['bare_nucleoli'].replace('?', np.nan, inplace=True)     # '?'를 NaN로 바꿈
df.dropna(subset=['bare_nucleoli'], axis=0, inplace=True)    # dropna() 함수로 NaN값 삭제
df['bare_nucleoli'] = df['bare_nucleoli'].astype('int64')   # '?'가 있던 열 --> object 자료형에서 int 자료형으로 변환

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id               683 non-null    int64
 1   clump            683 non-null    int64
 2   cell_size        683 non-null    int64
 3   cell_shape       683 non-null    int64
 4   adhesion         683 non-null    int64
 5   epithlial        683 non-null    int64
 6   bare_nucleoli    683 non-null    int64
 7   chromatin        683 non-null    int64
 8   normal_nucleoli  683 non-null    int64
 9   mitoses          683 non-null    int64
 10  class            683 non-null    int64
dtypes: int64(11)
memory usage: 64.0 KB


In [19]:
# 설명변수 = X, 목적변수 = y
X = df[['clump', 'cell_size', 'cell_shape', 'adhesion', 'epithlial', 
              'bare_nucleoli', 'chromatin', 'normal_nucleoli', 'mitoses']]
y = df['class']

# 정규화
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit(X).transform(X)

# 트레인 데이터 /테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print(X_train.shape, ' : ', X_test.shape)

(546, 9)  :  (137, 9)


In [ ]:
'''
결정트리 분류 모형
구조 : 컴퓨터 알고리즘에서 사용하는 트리(tree) 구조를 사용
      각 분기점(node)에는 분석대상의 속성(설명변수)들이 위치, 
      해당 속성이 갖는 값을 이용하여 새로운 가지(branch)를 만드는 방식
      - 각 분기점에서 최적의 속성을 선택할 때 해당 속성을 기준으로 분류한 값들이 구분되는 정도를 측정
      => 다른 종류의 값들이 섞여 있는 정도를 나타내는 entropy를 주로 활용
      => 이 entropy가 낮을수록 분류가 잘 되는 형태
      => entropy가 일정 수준 이하로 낮아질 때까지 과정을 반복
      
      # entropy란 쉽게 말하자면 불순도로서, 
      박스 안에 파란공이나 빨간공만 들어가 있을 때는 0, 반반 들어가있을 경우 최대값을 갖는다
'''

In [ ]:
# 결정트리 모델 학습 / 예측 / 평가

In [21]:
from sklearn.tree import DecisionTreeClassifier

# max_depth : 트리 레벨
# 트리레벨이 깊어지면(많아지면) 정확해지나, 너무 깊어지면 과대적합 발생

tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=5)
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)

In [22]:
print(y_pred[:10])
print('---------------------------------')
print(y_test.values[:10])

[4 4 2 4 4 4 2 2 4 4]
---------------------------------
[4 4 4 4 4 4 2 2 4 4]


In [23]:
# 성능 평가
from sklearn.metrics import confusion_matrix, classification_report

tree_matrix = confusion_matrix(y_test, y_pred)
print(tree_matrix)
print('-----------------')
tree_report = classification_report(y_test, y_pred)
print(tree_report)

[[83  6]
 [ 2 46]]
-----------------
              precision    recall  f1-score   support

           2       0.98      0.93      0.95        89
           4       0.88      0.96      0.92        48

    accuracy                           0.94       137
   macro avg       0.93      0.95      0.94       137
weighted avg       0.94      0.94      0.94       137



In [ ]:
# [과제] titanic 데이터를 이용해서 결정트리 모델로 학습시키고 예측, 평가